In [107]:
import random
import time
import tkinter as tk
from itertools import combinations

In [108]:
BOARD_SIZE = 9  # Size of the board (9x9 for standard Sudoku)
SUBGRID_SIZE = 3  # Size of the subgrids (3x3 for standard Sudoku)
EMPTY_CELLS = 42 # Number of empty cells in template
INITIAL_DELAY = 0.25  # Initial delay in seconds
REDUCED_DELAY = 0.05  # Reduced delay in seconds
CHANGE_DELAY_TIME = 15  # Time after which the delay changes (in seconds)

# List of colors for the cages
COLORS = ["#FFA07A", "#E9967A", "#F0E68C", "#98FB98", "#AFEEEE", "#DB7093", "#FFE4E1", "#D8BFD8", "#FFD700", "#B0C4DE"]

def is_valid(board, row, col, num):
    """Check if num can be placed at board[row][col]."""
    for i in range(BOARD_SIZE):
        if board[row][i] == num or board[i][col] == num:
            return False
    
    start_row, start_col = SUBGRID_SIZE * (row // SUBGRID_SIZE), SUBGRID_SIZE * (col // SUBGRID_SIZE)
    for i in range(start_row, start_row + SUBGRID_SIZE):
        for j in range(start_col, start_col + SUBGRID_SIZE):
            if board[i][j] == num:
                return False
    
    return True

def fill_board(board):
    """Fill the board with a valid Sudoku solution using backtracking."""
    numbers = list(range(1, BOARD_SIZE + 1))
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if board[row][col] == 0:
                random.shuffle(numbers)  # Shuffle numbers to introduce randomness
                for num in numbers:
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        if fill_board(board):
                            return True
                        board[row][col] = 0
                return False
    return True

def generate_sudoku_template(empty_cells=45):
    """Generate a Sudoku template with a certain number of cells left empty."""
    board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    fill_board(board)
    
    # Remove some numbers to create a Sudoku template
    removed = 0
    while removed < empty_cells:
        row, col = random.randint(0, BOARD_SIZE - 1), random.randint(0, BOARD_SIZE - 1)
        if board[row][col] != 0:
            board[row][col] = 0
            removed += 1
    
    return board

def create_gui(board, cages=None):
    """Create a tkinter GUI for displaying the Sudoku board."""
    root = tk.Tk()
    root.title("Killer Sudoku Solver")
    canvas = tk.Canvas(root, width=450, height=450)
    canvas.pack()

    cell_width = 450 // BOARD_SIZE
    cell_height = 450 // BOARD_SIZE

    for i in range(BOARD_SIZE + 1):
        line_width = 3 if i % SUBGRID_SIZE == 0 else 1
        canvas.create_line(0, i * cell_height, 450, i * cell_height, width=line_width)
        canvas.create_line(i * cell_width, 0, i * cell_width, 450, width=line_width)

    cells = [[None for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            x1 = j * cell_width
            y1 = i * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            cell_text = str(board[i][j]) if board[i][j] != 0 else ''
            font = ("Arial", 16, "bold") if board[i][j] != 0 else ("Arial", 16)
            fill = "black" if board[i][j] != 0 else "black"
            cells[i][j] = canvas.create_text((x1 + x2) / 2, (y1 + y2) / 2, text=cell_text, font=font, fill=fill)
    
    if cages:
        for index, (cage, cage_sum) in enumerate(cages):
            color = COLORS[index % len(COLORS)]
            for (row, col) in cage:
                x1 = col * cell_width
                y1 = row * cell_height
                x2 = x1 + cell_width
                y2 = y1 + cell_height
                canvas.create_rectangle(x1, y1, x2, y2, fill=color, stipple="gray50")
            x1 = min(cell[1] for cell in cage) * cell_width
            y1 = min(cell[0] for cell in cage) * cell_height
            canvas.create_text(x1 + 5, y1 + 5, text=str(cage_sum), anchor=tk.NW, font=("Arial", 10, "bold"))

    return root, canvas, cells


def update_gui(canvas, cells, board):
    """Update the GUI to reflect the current state of the board."""
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            cell_text = str(board[i][j]) if board[i][j] != 0 else ''
            canvas.itemconfig(cells[i][j], text=cell_text)
    canvas.update()

def ai_solve_process(board, canvas, cells, start_time, delay=INITIAL_DELAY):
    """Solve the Sudoku board with real-time visualization."""
    current_time = time.time()
    elapsed_time = current_time - start_time

    # Change delay after 20 seconds
    if elapsed_time > CHANGE_DELAY_TIME:
        delay = REDUCED_DELAY

    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if board[row][col] == 0:
                for num in range(1, BOARD_SIZE + 1):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        colors[row][col] = "black"
                        update_gui(canvas, cells, board, colors)  # Update the GUI to show progress
                        time.sleep(delay)  # Wait for the specified delay
                        if ai_solve_process(board, canvas, cells, start_time, delay):
                            return True
                        colors[row][col] = "red"
                        update_gui(canvas, cells, board, colors)  # Update the GUI to show backtracking
                        time.sleep(delay)  # Wait for the specified delay
                        board[row][col] = 0
                        colors[row][col] = "black"
                        update_gui(canvas, cells, board, colors)
                return False
            
    # After solving, set all added numbers to green
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if cells[i][j] and canvas.itemcget(cells[i][j], 'fill') == 'black':
                colors[i][j] = "green"
    update_gui(canvas, cells, board, colors)
    return True




AI solved answer:

In [109]:
def generate_killer_sudoku():
    """Generate a Killer Sudoku template with cages and their sums."""
    board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    
    # Generate random cages
    cages = []
    used = set()
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if (i, j) not in used:
                cage_size = random.randint(2, 5)  # Random cage size between 2 and 5
                cage = [(i, j)]
                used.add((i, j))
                for _ in range(cage_size - 1):
                    possible_cells = [(i + x, j + y) for x, y in [(-1,0), (1,0), (0,-1), (0,1)] if (i + x, j + y) not in used and 0 <= i + x < BOARD_SIZE and 0 <= j + y < BOARD_SIZE]
                    if possible_cells:
                        cell = random.choice(possible_cells)
                        cage.append(cell)
                        used.add(cell)
                cage_sum = random.randint(len(cage) + 1, 9 * len(cage))  # Random sum for the cage
                cages.append((cage, cage_sum))
    
    return board, cages

def is_valid_killer(board, row, col, num, cages):
    """Check if num can be placed at board[row][col] while respecting Sudoku and cage constraints."""
    for i in range(BOARD_SIZE):
        if board[row][i] == num or board[i][col] == num:
            return False
    
    start_row, start_col = SUBGRID_SIZE * (row // SUBGRID_SIZE), SUBGRID_SIZE * (col // SUBGRID_SIZE)
    for i in range(start_row, start_row + SUBGRID_SIZE):
        for j in range(start_col, start_col + SUBGRID_SIZE):
            if board[i][j] == num:
                return False

    for cage, cage_sum in cages:
        if (row, col) in cage:
            current_sum = sum(board[x][y] for x, y in cage if board[x][y] != 0) + num
            if current_sum > cage_sum:
                return False
            if board[row][col] == 0 and any(board[x][y] == num for x, y in cage):
                return False

    # Forward checking for the same row and column
    for i in range(BOARD_SIZE):
        if board[row][i] == 0 and i != col:
            if not any(board[row][j] == num for j in range(BOARD_SIZE)):
                return False
        if board[i][col] == 0 and i != row:
            if not any(board[j][col] == num for j in range(BOARD_SIZE)):
                return False

    # Forward checking within the cage
    for i, j in cage:
        if board[i][j] == 0:
            possible_values = set(range(1, BOARD_SIZE + 1))
            for x, y in cage:
                if x != row or y != col:
                    possible_values.discard(board[x][y])
            if not possible_values:
                return False
    
    return True

def ai_solve_killer_process(board, canvas, cells, start_time, delay, cages):
    """Solve the Killer Sudoku board with real-time visualization."""
    current_time = time.time()
    elapsed_time = current_time - start_time

    # Change delay after 20 seconds
    if elapsed_time > CHANGE_DELAY_TIME:
        delay = REDUCED_DELAY

    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if board[row][col] == 0:
                for num in range(1, BOARD_SIZE + 1):
                    if is_valid_killer(board, row, col, num, cages):
                        board[row][col] = num
                        colors[row][col] = "black"
                        update_gui(canvas, cells, board, colors)  # Update the GUI to show progress
                        time.sleep(delay)  # Wait for the specified delay
                        if ai_solve_killer_process(board, canvas, cells, start_time, delay, cages):
                            return True
                        colors[row][col] = "red"
                        update_gui(canvas, cells, board, colors)  # Update the GUI to show backtracking
                        time.sleep(delay)  # Wait for the specified delay
                        board[row][col] = 0
                        colors[row][col] = "black"
                        update_gui(canvas, cells, board, colors)
                return False

    # After solving, set all added numbers to green
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if cells[i][j] and canvas.itemcget(cells[i][j], 'fill') == 'black':
                colors[i][j] = "green"
    update_gui(canvas, cells, board, colors)
    return True

In [110]:
# Generate a Killer Sudoku template
sudoku_template, cages = generate_killer_sudoku()

# Create and start the GUI
root, canvas, cells = create_gui(sudoku_template, cages)
colors = [["black" if sudoku_template[i][j] != 0 else "black" for j in range(BOARD_SIZE)] for i in range(BOARD_SIZE)]

In [111]:
def start_solving():
    start_time = time.time()
    if ai_solve_killer_process(sudoku_template, canvas, cells, start_time, INITIAL_DELAY, cages):
        print("Killer Sudoku solved by AI:")
    else:
        print("No solution exists for the given Killer Sudoku.")

root.after(1000, start_solving)  # Start solving after a 1-second delay to allow the GUI to load
root.mainloop()

No solution exists for the given Killer Sudoku.


In [121]:
def create_custom_killer_sudoku(board, cages=None):
    """Create a custom Killer Sudoku board based on the provided cages."""
    # Create and start the GUI
    root, canvas, cells = create_gui(board, cages)
    update_gui(canvas, cells, board)
    root.mainloop()

def main():
    # Define the Killer Sudoku board
    sudoku_board = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
    
    # Define the cages and their sums
    custom_cages = [
        ([(0, 0), (0, 1)], 3),
        ([(0, 2), (0, 3), (0, 4)], 15),
        ([(0, 5), (1, 5), (1, 4), (2, 4)], 22),
        # Add more cages as needed
    ]
    
    create_custom_killer_sudoku(sudoku_board, custom_cages)

if __name__ == "__main__":
    main()